# How to Detect Duplicates Using Hashing

#TODO: Write me..

In [ ]:
from binascii import hexlify
from datetime import datetime
import hashlib
from hmac import compare_digest
import json
import os

import requests as r

## Hashing Introduction

### What is a hash?

A hash function is a deterministic mathematical function that takes an input of any length and content (e.g. letters, numbers, and symbols) and uses a formula to produce an output of a specific length.  

### What can be hashed?

A hash can be created using nearly any form of digitial content: a document, image, song, etc.


reference: [What is hashing?](https://medium.com/tech-tales/what-is-hashing-6edba0ebfa67)

In [ ]:
# Initialize the hasher
hasher = hashlib.sha256()

# let's hash something
hasher.update(b"I am a fun hash!")

print("I am a fun hash!: " + hasher.hexdigest())

hasher.update(b"To be or not to be")

print("I am a fun hash!  To be or not to be: " + hasher.hexdigest())

In [ ]:
# MD5 (Messsage Digest) Hashing Examples

hasher = hashlib.md5()

shakespeare_sonnet = """
Shall I compare thee to a summer’s day?
Thou art more lovely and more temperate:
Rough winds do shake the darling buds of May,
And summer’s lease hath all too short a date;
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimm'd;
And every fair from fair sometime declines,
By chance or nature’s changing course untrimm'd;
But thy eternal summer shall not fade,
Nor lose possession of that fair thou ow’st;
Nor shall death brag thou wander’st in his shade,
When in eternal lines to time thou grow’st:
So long as men can breathe or eyes can see,
So long lives this, and this gives life to thee.
"""

hasher.update(shakespeare_sonnet.encode('utf-8'))

print(hasher.hexdigest())

In [ ]:
# Hashing an image

paris_picture = 'https://lonelyplanetimages.imgix.net/mastheads/GettyImages-500759045_super.jpg?sharp=10&vib=20&w=1200'

image_binary = r.get(paris_picture).content

print('Image size: ' + str(len(image_binary)))

print(b'some of the image content:' + image_binary[0:10])

picture_hash = hashlib.md5(image_binary)

print('Image hash: ' + picture_hash.hexdigest())

## Example:  How to detect duplicate dictionaries

Suppose Alice sends Bob a series of JSON objects over the wire, but Alice is forgetful and sends Bob multiple JSON blobs containing the same data.  Bob doesn't want to have to read Alice's messages multiple times, so he needs a way to figure out if they're duplicate messages.

In [ ]:
alices_messages = [
    {'message': 'Hello, Bob!', 'timestamp': datetime.now()},
    {'message': 'How are you doing, Bob?', 'timestamp': datetime.now()},
    {'id': 12454432, 'message': 'I see you Bob!', 'timestamp': datetime.now()},
    {'message': 'How are you doing, Bob?', 'timestamp': datetime.now()},
]

print(alices_messages)


# bob gets the message batch

for message in alices_messages:
    encoded_message = message.get('message').encode('utf-8')
    message_hash = hashlib.sha256(encoded_message).hexdigest()
    print(f'{encoded_message}: {message_hash}')

In [ ]:
# Bob notices Alice is sending duplicate messages, and decides to create a dictionary to filter them.
duplicate_timestamp = datetime.now()

alices_messages = [
    {'message': 'Hello, Bob!', 'timestamp': datetime.now(), 'weather': 'cloudy with a chance of meatballs'},
    {'message': 'How are you doing, Bob?', 'timestamp': duplicate_timestamp},
    {'id': 12454432, 'message': 'I see you Bob!', 'timestamp': datetime.now()},
    {'message': 'How are you doing, Bob?', 'timestamp': duplicate_timestamp},
]

unique_messages = {}

for message in alices_messages:
    encoded_message = json.dumps(message, sort_keys=True, default=str)
    print(f'encoded message: {encoded_message}')
    message_hash = hashlib.sha256(encoded_message.encode('utf-8')).hexdigest()
    print(f'hash: {message_hash}')
    
    # check to see if message exists, and if not add it to the docket
    if message_hash not in unique_messages:
        unique_messages.update({message_hash: message})
    else:
        print('Silly Alice, she sent a duplicate message!')

In [ ]:
print(unique_messages)

## Cryptographically Secure Hashing

### Overview

In [ ]:
# Generate a key
KEY_LENGTH = 30
AUTH_SIZE = 16

secret_key = hexlify(os.urandom(KEY_LENGTH))


def sign(cookie):
    h = hashlib.blake2b(digest_size=AUTH_SIZE, key=secret_key)
    h.update(cookie.encode('utf-8'))
    return h.hexdigest().encode('utf-8')


def verify(cookie, sig):
    good_sig = sign(cookie)
    return compare_digest(good_sig, sig)

In [ ]:
cookie = {'user': 'alice', 'auth_mode': 'token'}
cookie_str = json.dumps(cookie, sort_keys=True)
sig = sign(cookie_str)

print(cookie_str)
print(sig)

print(verify(cookie_str, sig))

invalid_cookie = {'user': 'admin', 'auth_mode': 'token'}
invalid_cookie_str = json.dumps(invalid_cookie, sort_keys=True)
print(sign(invalid_cookie_str))

print(verify(invalid_cookie_str, sig))

# Detecting Duplicate Files

In [ ]:
import tempfile
import shutil
import random

def hash_file(file_path, block_size=1024):
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as f:
        hasher.update(f.read(block_size))
    return hasher.hexdigest().encode('utf-8')

In [ ]:
# Generate some random files in a temp directory.
temp_dir = tempfile.mkdtemp()

print('Temporary Directory: ' + temp_dir)

In [ ]:
f = open(os.path.join(temp_dir, 'f.txt'), 'wb')
f.write(os.urandom(2048))
f.close()

g = open(os.path.join(temp_dir, 'g.txt'), 'wb')
g.write(os.urandom(2048**2))
g.close()

f_hash = hash_file(os.path.join(temp_dir, 'f.txt'))
print(f_hash)

g_hash = hash_file(os.path.join(temp_dir, 'g.txt'))
print(g_hash)

compare_digest(f_hash, g_hash)

In [ ]:
# Generate random files.
FILE_NUM = 1000

random_data = (os.urandom(2048) for _ in range(0, FILE_NUM))
for indx, random_datum in enumerate(random_data):
    with open(os.path.join(temp_dir, '{}.txt'.format(str(indx))), 'wb') as f:
        f.write(random_datum)

# generate 1 duplicate file
random_file = os.path.join(temp_dir, '{}.txt'.format(str(random.randint(0, FILE_NUM - 1))))

with open(random_file, 'rb') as f:
    with open(os.path.join(temp_dir, 'dup_file.txt'), 'wb') as g:
        g.write(f.read())
        
print(random_file)

In [ ]:
# Find and list duplicate files.
# Algorithm is ~ O(n^2)

# Iterate over files in temp_dir
for root, directory, files in os.walk(temp_dir):
    # pick a file and iterate over it
    for o_file in files:
        o_fp = os.path.join(root, o_file)
        # print('checking: {}'.format(o_fp))
        outer_hash = hash_file(o_fp)
        for i_file in files:
            if o_file != i_file:
                i_fp = os.path.join(root, i_file)
                inner_hash = hash_file(i_fp)
                if compare_digest(outer_hash, inner_hash):
                    print('  found duplicate files: {} and {}'.format(o_fp, i_fp))

In [ ]:
# Clean up temp_dir

shutil.rmtree(temp_dir)

os.path.exists(temp_dir)